# Entraîner un classificateur simple sur les caractéristiques extraites

Les algorithmes d'apprentissage automatique ne sont pas correctement équipés pour fonctionner avec des tenseurs, ce qui leur interdit d'apprendre directement à partir d'images. Cependant, en utilisant des réseaux pré-entraînés comme extracteurs de caractéristiques, nous comblons cet écart, nous permettant d'accéder à la puissance d'algorithmes largement populaires et testés au combat tels que la régression logistique, les arbres de décision et les machines à vecteurs de support.

Dans cette recette, nous allons utiliser les fonctionnalités que nous avons générées dans la recette précédente (au format HDF5) pour entraîner un détecteur d'orientation d'image à corriger les degrés de rotation d'une image, pour restaurer son état d'origine.

Comme nous l'avons mentionné dans l'introduction de ce reipce, nous utiliserons le jeu de données features.hdf5 que nous avons généré dans la recette précédente, qui contient des informations codées sur les images pivotées du jeu de données Stanford Cars. Nous supposons que l'ensemble de données se trouve à l'emplacement suivant : /content/car_ims_rotated/features.hdf5

Suivez ces étapes pour terminer cette recette :

**1.** Importez les packages requis :

In [13]:
import pathlib

import h5py
from sklearn.linear_model import LogisticRegressionCV
from sklearn.metrics import classification_report

**2.** Chargez le jeu de données au format HDF5 :

In [20]:
dataset_path = str(pathlib.Path('/content/car_ims_rotated/features.hdf5'))
                   
db = h5py.File(dataset_path, 'r')

**3.** L'ensemble de données étant trop volumineux, nous ne travaillerons qu'avec 50 % des données. Le bloc suivant divise à la fois les entités et les étiquettes en deux :

In [21]:
SUBSET_INDEX = int(db['labels'].shape[0] * 0.5)
features = db['features'][:SUBSET_INDEX]
labels = db['labels'][:SUBSET_INDEX]

**4.** Prenez les premiers 80 % des données pour entraîner le modèle et les 20 % restants pour l'évaluer plus tard :

In [23]:
TRAIN_PROPORTION = 0.8
SPLIT_INDEX = int(len(labels) * TRAIN_PROPORTION)

X_train, y_train = (features[:SPLIT_INDEX],
                    labels[:SPLIT_INDEX])
X_test, y_test = (features[SPLIT_INDEX:],
                  labels[SPLIT_INDEX:])

**5.** Former un modèle de régression logistique à validation croisée sur l'ensemble d'apprentissage. LogisticRegressionCV trouvera le meilleur paramètre C en utilisant la validation croisée :

In [24]:
model = LogisticRegressionCV(n_jobs=-1)
model.fit(X_train, y_train)

LogisticRegressionCV(Cs=10, class_weight=None, cv=None, dual=False,
                     fit_intercept=True, intercept_scaling=1.0, l1_ratios=None,
                     max_iter=100, multi_class='auto', n_jobs=-1, penalty='l2',
                     random_state=None, refit=True, scoring=None,
                     solver='lbfgs', tol=0.0001, verbose=0)

Notez que n_jobs=-1 signifie que nous utiliserons tous les cœurs disponibles pour trouver le meilleur modèle en parallèle. Vous pouvez ajuster cette valeur en fonction de la capacité de votre matériel

**6.** Évaluez le modèle sur l'ensemble de test. Nous allons calculer un rapport de classification pour obtenir une vue précise des performances du modèle :

In [42]:
predictions = model.predict(X_test)
#report = classification_report(y_test, predictions,
#                               target_names=db['label_names'])
report = classification_report(y_test, predictions,
                               target_names=['0', '90', '180', '270'])
print(report)

              precision    recall  f1-score   support

           0       1.00      1.00      1.00       226
          90       0.98      0.98      0.98       184
         180       0.99      1.00      1.00       202
         270       0.99      0.98      0.98       203

    accuracy                           0.99       815
   macro avg       0.99      0.99      0.99       815
weighted avg       0.99      0.99      0.99       815



Le modèle fait un bon travail de discrimination entre les quatre classes, atteignant une précision globale de 99% sur l'ensemble de test !

**7.** Enfin, fermez le chier HDF5 pour libérer des ressources :

In [43]:
db.close()

Nous venons d'entraîner un modèle de régression logistique très simple pour détecter le degré de rotation d'une image. Pour y parvenir, nous avons tiré parti des fonctionnalités riches et expressives que nous avons extraites à l'aide d'un réseau VGG16 pré-entraîné sur ImageNet.

Étant donné que ces données sont trop volumineuses et que les algorithmes d'apprentissage automatique de scikit-learn fonctionnent avec l'intégralité des données en une seule fois (plus précisément, la plupart d'entre eux ne peuvent pas fonctionner par lots), nous n'avons utilisé que 50 % des fonctionnalités et des étiquettes, en raison de contraintes de mémoire.

Après quelques minutes, nous avons obtenu une performance incroyable de 99% sur l'ensemble de test. De plus, en analysant le rapport de classification, nous pouvons voir que le modèle est très conant dans ses prédictions, atteignant un score F1 d'au moins 0,99 dans les quatre cas